In [1]:
from sklearn.neighbors import NearestNeighbors
import numpy as np
import pandas as pd
from pathlib import Path
import os
from plotnine import *
%matplotlib inline
import matplotlib.pyplot as plt
import seaborn as sns
from descartes import PolygonPatch
import alphashape
from shapely.geometry import Point
from shapely.geometry import Polygon
from shapely.geometry import Point
import math
from scipy.stats import kde
import hdbscan
from sklearn.cluster import DBSCAN
from scipy.spatial import Delaunay
from collections import defaultdict
from itertools import permutations
import plotly.figure_factory as ff
import itertools

#Data-Wrangling with DBSCAN results
#clean, extract, and export useful data
folder_location="C:/Data/raw"
dir=Path(folder_location)
os.chdir(folder_location)
for f in os.listdir(dir):
    if f.endswith(".csv"):
        df = pd.read_csv(f, sep=",")
        
        Data = {"Pt":[],"Phenotype":[], "Cluster":[], "X":[], "Y":[]} 
        Data["Pt"] = df["Path"][1].split("\\")[-3]
        Data["Cluster"] = df["DBSCAN Cluster"]
        
        for column in df.columns.unique():
            if "Complex Complex Phenotype" in column:
                Data["Phenotype"] = df["Complex Complex Phenotype"]
            elif "Complex.Complex.Phenotype" in column:
                Data["Phenotype"] = df["Complex.Complex.Phenotype"]  
            elif "Complex.Phenotype" in column:
                Data["Phenotype"] = df["Complex.Phenotype"]
            elif "Complex Phenotype" in column:
                Data["Phenotype"] = df["Complex Phenotype"]                
            elif "Phenotype" in column:
                Data["Phenotype"] = df["Phenotype"]
                
        for column in df.columns.unique():
            if "Cell X Position NEW" in column:
                Data["X"] = df["Cell X Position NEW"].round(1) 
                Data["Y"] = df["Cell Y Position NEW"].round(1)
            elif "Cell.X.Position" in column:
                Data["X"] = df["Cell.X.Position"].round(1) 
                Data["Y"] = df["Cell.Y.Position"].round(1)
                
        Data_df=pd.DataFrame(Data)
        
        #This script is necessary because every sample has different numbers of "Belong to DBSCAN..." columns.
        Belong =[]
        for j in df.columns:
            if "Belong to DBSCAN" in j:
                Belong.append(j)
                
        for b in np.arange(0,len(Belong)):
            Data_df["Belong to DBSCAN cluster%s"%b] = df[["Belong to DBSCAN cluster%s"%b]]
            
        Data_df_clean = Data_df.replace({"Phenotype":{"CD8+OthersOthers":"CD8+",
                                              "CD141+OthersOthers":"CD141+PD-L1-",
                                              "OthersOthersOthers":"Others",
                                              "CD141+PD-L1Others":"CD141+PD-L1+",
                                              "OthersPD-L1Others":"Others",
                                              "CD8+PD-L1Others":"CD8+",
                                              "OthersOthersCD1a+":"CD1a+PD-L1-",
                                              "CD141+PD-L1CD1a+":"CD1a+PD-L1+",
                                              "OthersPD-L1CD1a+":"CD1a+PD-L1+",
                                              "CD141+OthersCD1a+":"CD1a+PD-L1-",
                                              "CD8+OthersCD1a+":"CD8+",
                                              "CD8+PD-L1CD1a+":"CD8+",
                                              "OthersOthersCD1a":"CD1a+PD-L1-",
                                              "CD141+PD-L1CD1a":"CD1a+PD-L1+",
                                              "OthersPD-L1CD1a":"CD1a+PD-L1+",
                                              "CD141+OthersCD1a":"CD1a+PD-L1-",
                                              "CD8+OthersCD1a":"CD8+",
                                              "CD8+PD-L1CD1a":"CD8+",
                                              "OthersPD-L1-":"Others",
                                              "OthersPD-L1+":"Others",
                                              "CD8+PD-L1-":"CD8+",
                                              "CD8+PD-L1+":"CD8+",
                                              "OthersPDL1-":"Others",
                                              "OthersPDL1+":"Others",
                                              "CD8+PDL1-":"CD8+",
                                              "CD8+PDL1+":"CD8+",
                                              "CD141+PDL1-":"CD141+PD-L1-",
                                              "CD141+PDL1+":"CD141+PD-L1+",
                                              "CD1a+PDL1-":"CD1a+PD-L1-",
                                              "CD1a+PDL1+":"CD1a+PD-L1+",
                                               "CD141+Others":"CD141+PD-L1-",
                                               "OthersOthers":"Others",
                                               "CD8+Others":"CD8+",
                                               "CD1a+Others":"CD1a+PD-L1-",
                                               "CD141+PD-L1":"CD141+PD-L1+",
                                               "OthersPD-L1":"Others",
                                               "CD8+PD-L1":"CD8+",
                                               "CD1a+PD-L1":"CD1a+PD-L1+",
                                                "CD8+OthersOthers":"CD8+",
                                                "OthersPD-L1Others":"Others",
                                                "CD8+PD-L1Others":"CD8+",
                                                "CD141+PD-L1CD1a+":"CD1a+PD-L1+",
                                                "OthersOthersCD1a+":"CD1a+PD-L1-",
                                                "CD141+OthersCD1a+":"CD1a+PD-L1-",
                                                "CD8+PD-L1CD1a+":"CD8+",
                                                "CD8+OthersCD1a+":"CD8+"}})
     

        Data_df_clean.to_csv("C:/Data/DCs1_res/Res_"+
                            df["Path"][1].split("\\")[-3]+".txt")

# Import and concat to one: 
file = []
folder_location="C:/Data/DCs1_res"
dir=Path(folder_location)
os.chdir(folder_location)
for f in os.listdir(dir):
    if f.endswith(".txt"):
        file.append(pd.read_csv(f, sep=","))
        df_DCs = pd.concat(file)
df_DCs.to_csv("C:/Data/DCs1_res/Res_all.txt")